In [1]:
import pandas as pd
import json
import requests
import time
import numpy as np
from functools import reduce

## Introduction

The purpose of this notebook is to show the steps for data prepration for the prediction task.

## Data Collection
We get our raw data from Alpha Vantage APIs as it's refined and free of cost.
The data is got in form of JSON object which is stored as a pandas dataframe in a dictionary with symbol name as the key

In [3]:
# commonly used variables

symbol = "AAPL";
apikey = "V1SZPLE0U8CCXSFW.";
datatype = "json";
url = "https://www.alphavantage.co/query";
outputsize = "full";
interval = "daily";
timeperiod = "10";
seriestype = "close";
index = "date";

In [4]:
# Time Series Data

parameters = {'function':'TIME_SERIES_DAILY_ADJUSTED', 'outputsize':outputsize,'datatype':datatype,'apikey':apikey,'symbol':symbol}

response = requests.get(url, params = parameters);
data = response.json()

AAPL_TS_data = pd.DataFrame(data["Time Series (Daily)"]).T

AAPL_TS_data.reset_index(inplace=True);
AAPL_TS_data.rename(columns={'index': 'date'}, inplace=True);

AAPL_TS_data.rename(columns={'1. open': 'open'}, inplace=True);
AAPL_TS_data.rename(columns={'2. high': 'high'}, inplace=True);
AAPL_TS_data.rename(columns={'3. low': 'low'}, inplace=True);
AAPL_TS_data.rename(columns={'4. close': 'close'}, inplace=True);
AAPL_TS_data.rename(columns={'5. adjusted close': 'adjusted close'}, inplace=True);
AAPL_TS_data.rename(columns={'6. volume': 'volume'}, inplace=True);
AAPL_TS_data.rename(columns={'7. dividend amount': 'dividend amount'}, inplace=True);
AAPL_TS_data.rename(columns={'8. split coefficient': 'split coefficient'}, inplace=True);

# Let's add another column to the frame which is the adjusted close data shifted by 10 days which is what we want to predict

AAPL_TS_data["Target"] = AAPL_TS_data["adjusted close"].shift(-10);

# Now that we donot have the data of the future we need to clip the dataframe where the target is not defined 
AAPL_TS_data = AAPL_TS_data.dropna()

display(AAPL_TS_data.tail())
display(AAPL_TS_data.info())

,date,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,Target
5225,2018-10-08,222.2100,224.8000,220.2000,223.7700,223.7700,29663923,0.0000,1.0000,220.6500
5226,2018-10-09,223.6400,227.2700,222.2462,226.8700,226.8700,26891029,0.0000,1.0000,222.7300
5227,2018-10-10,225.4600,226.3500,216.0500,216.3600,216.3600,41990554,0.0000,1.0000,215.0900
5228,2018-10-11,214.5200,219.5000,212.3200,214.4500,214.4500,53124392,0.0000,1.0000,219.8000
5229,2018-10-12,220.4200,222.8800,216.8400,222.1100,222.1100,40337851,0.0000,1.0000,216.3000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5230 entries, 0 to 5229
Data columns (total 10 columns):
date                 5230 non-null object
open                 5230 non-null object
high                 5230 non-null object
low                  5230 non-null object
close                5230 non-null object
adjusted close       5230 non-null object
volume               5230 non-null object
dividend amount      5230 non-null object
split coefficient    5230 non-null object
Target               5230 non-null object
dtypes: object(10)
memory usage: 449.5+ KB


None

In [5]:
#ADX Data

parameters = {'function':'ADX', 'interval':interval,'time_period':timeperiod,'datatype':datatype,'apikey':apikey,'symbol':symbol}

response = requests.get(url, params = parameters);
data = response.json()

AAPL_ADX_data = pd.DataFrame(data["Technical Analysis: ADX"]).T

AAPL_ADX_data.reset_index(inplace=True);
AAPL_ADX_data.rename(columns={'index': 'date'}, inplace=True);


display(AAPL_ADX_data.head())
display(AAPL_ADX_data.info())

,date,ADX
0,1998-01-30,50.0186
1,1998-02-02,45.0540
2,1998-02-03,40.8746
3,1998-02-04,37.1132
4,1998-02-05,33.7280


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5221 entries, 0 to 5220
Data columns (total 2 columns):
date    5221 non-null object
ADX     5221 non-null object
dtypes: object(2)
memory usage: 81.7+ KB


None

In [7]:
#RSI Data

parameters = {'function':'RSI', 'interval':interval,'time_period':timeperiod,'series_type':seriestype,'datatype':datatype,'apikey':apikey,'symbol':symbol}

response = requests.get(url, params = parameters);
data = response.json()

AAPL_RSI_data = pd.DataFrame(data["Technical Analysis: RSI"]).T

AAPL_RSI_data.reset_index(inplace=True);
AAPL_RSI_data.rename(columns={'index': 'date'}, inplace=True);

display(AAPL_RSI_data.head())
display(AAPL_RSI_data.info())

,date,RSI
0,1998-01-16,65.8809
1,1998-01-20,67.0176
2,1998-01-21,65.5614
3,1998-01-22,67.3482
4,1998-01-23,68.6760


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5230 entries, 0 to 5229
Data columns (total 2 columns):
date    5230 non-null object
RSI     5230 non-null object
dtypes: object(2)
memory usage: 81.8+ KB


None

In [8]:
#SMA_data

parameters = {'function':'SMA', 'interval':interval,'time_period':timeperiod,'series_type':seriestype,'datatype':datatype,'apikey':apikey,'symbol':symbol}

response = requests.get(url, params = parameters);
data = response.json()

AAPL_SMA_data = pd.DataFrame(data["Technical Analysis: SMA"]).T
AAPL_SMA_data.reset_index(inplace=True);
AAPL_SMA_data.rename(columns={'index': 'date'}, inplace=True);

display(AAPL_SMA_data.head())
display(AAPL_SMA_data.info())

,date,SMA
0,1998-01-15,18.1640
1,1998-01-16,18.4200
2,1998-01-20,18.7380
3,1998-01-21,18.7350
4,1998-01-22,18.9100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5231 entries, 0 to 5230
Data columns (total 2 columns):
date    5231 non-null object
SMA     5231 non-null object
dtypes: object(2)
memory usage: 81.8+ KB


None

In [10]:
# MACD_data

parameters = {'function':'MACD', 'interval':interval,'series_type':seriestype,'datatype':datatype,'apikey':apikey,'symbol':symbol}

response = requests.get(url, params = parameters);
data = response.json()

AAPL_MACD_data = pd.DataFrame(data["Technical Analysis: MACD"]).T

AAPL_MACD_data.reset_index(inplace=True);
AAPL_MACD_data.rename(columns={'index': 'date'}, inplace=True);

display(AAPL_MACD_data.head())
display(AAPL_MACD_data.info())

,date,MACD,MACD_Hist,MACD_Signal
0,1998-02-20,0.4518,0.1604,0.2913
1,1998-02-23,0.5519,0.2084,0.3435
2,1998-02-24,0.6289,0.2283,0.4005
3,1998-02-25,0.7617,0.2890,0.4728
4,1998-02-26,0.9521,0.3835,0.5686


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5207 entries, 0 to 5206
Data columns (total 4 columns):
date           5207 non-null object
MACD           5207 non-null object
MACD_Hist      5207 non-null object
MACD_Signal    5207 non-null object
dtypes: object(4)
memory usage: 162.8+ KB


None

## Structuring the data
Now that we have all the data we can start with the preprocessing of the same.
let's define the fields that we are interested in.

we need below details for every stock

+ simple moving average
+ ADX
+ RSI
+ MACD
+ volume
+ high
+ low
+ adjusted close

And having all this info we would be predicting what the price would be 10 days down the line.

#### Let's first create the single data frame and split that into Train, Test and Validation data.

In [11]:
All_frames = [AAPL_ADX_data,AAPL_SMA_data,AAPL_RSI_data,AAPL_MACD_data,AAPL_TS_data];

# Luckily this cool function reduces the final merged dataframe dropping any of the rows where the values were not present.
# Thus we get a clean dataframe to use which is fully cleaned.

Merged_frames = reduce(lambda  left,right: pd.merge(left,right), All_frames)

display(Merged_frames.head())
display(Merged_frames.info())

,date,ADX,SMA,RSI,MACD,MACD_Hist,MACD_Signal,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,Target
0,1998-02-20,35.8066,19.5620,62.2914,0.4518,0.1604,0.2913,20.5000,20.5600,19.8100,20.0000,0.6330,2905500,0.0000,1.0000,0.7735
1,1998-02-23,38.2897,19.8370,72.2154,0.5519,0.2084,0.3435,20.1200,21.6200,20.0000,21.2500,0.6726,4263300,0.0000,1.0000,0.7200
2,1998-02-24,40.5245,20.0490,72.6000,0.6289,0.2283,0.4005,21.3100,21.3700,20.7500,21.3100,0.6745,4076700,0.0000,1.0000,0.7615
3,1998-02-25,43.7290,20.3360,78.1904,0.7617,0.2890,0.4728,21.3100,22.7500,20.9400,22.3100,0.7061,6363100,0.0000,1.0000,0.8267
4,1998-02-26,47.0657,20.7860,82.8240,0.9521,0.3835,0.5686,22.3100,23.5600,21.8700,23.5000,0.7438,5313700,0.0000,1.0000,0.8546


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5197 entries, 0 to 5196
Data columns (total 16 columns):
date                 5197 non-null object
ADX                  5197 non-null object
SMA                  5197 non-null object
RSI                  5197 non-null object
MACD                 5197 non-null object
MACD_Hist            5197 non-null object
MACD_Signal          5197 non-null object
open                 5197 non-null object
high                 5197 non-null object
low                  5197 non-null object
close                5197 non-null object
adjusted close       5197 non-null object
volume               5197 non-null object
dividend amount      5197 non-null object
split coefficient    5197 non-null object
Target               5197 non-null object
dtypes: object(16)
memory usage: 690.2+ KB


None

In [29]:
# And now for the final touch and creating the Training, Validation and Test sets

total_length = Merged_frames.shape[0];

Train_set = Merged_frames.iloc[:5000];
Validation_set = Merged_frames.iloc[5000:5100];
Test_set = Merged_frames.iloc[5100:total_length];

Train_labels = Train_set[['Target']].copy()
Train_data = Train_set.drop(['Target','date'], axis=1);

Validation_labels = Validation_set[['Target']].copy();
Validation_data = Validation_set.drop(['Target','date'], axis=1);

Test_labels = Test_set[['Target']].copy();
Test_data = Test_set.drop(['Target','date'], axis=1);


display(Train_labels.head())
display(Train_data.head())


display(Validation_labels.head())
display(Validation_data.head())

display(Test_labels.head())
display(Test_data.head())


Train_data.to_csv("..\data\Train_data.csv", encoding='utf-8');
Train_labels.to_csv("..\data\Train_labels.csv", encoding='utf-8');
Validation_data.to_csv("..\data\Validation_data.csv", encoding='utf-8');
Validation_labels.to_csv("..\data\Validation_labels.csv", encoding='utf-8');
Test_data.to_csv("..\data\Test_data.csv", encoding='utf-8');
Test_labels.to_csv("..\data\Test_labels.csv", encoding='utf-8');

,Target
0,0.7735
1,0.7200
2,0.7615
3,0.8267
4,0.8546


,ADX,SMA,RSI,MACD,MACD_Hist,MACD_Signal,open,high,low,close,adjusted close,volume,dividend amount,split coefficient
0,35.8066,19.5620,62.2914,0.4518,0.1604,0.2913,20.5000,20.5600,19.8100,20.0000,0.6330,2905500,0.0000,1.0000
1,38.2897,19.8370,72.2154,0.5519,0.2084,0.3435,20.1200,21.6200,20.0000,21.2500,0.6726,4263300,0.0000,1.0000
2,40.5245,20.0490,72.6000,0.6289,0.2283,0.4005,21.3100,21.3700,20.7500,21.3100,0.6745,4076700,0.0000,1.0000
3,43.7290,20.3360,78.1904,0.7617,0.2890,0.4728,21.3100,22.7500,20.9400,22.3100,0.7061,6363100,0.0000,1.0000
4,47.0657,20.7860,82.8240,0.9521,0.3835,0.5686,22.3100,23.5600,21.8700,23.5000,0.7438,5313700,0.0000,1.0000


,Target
5000,177.2290
5001,176.4381
5002,174.9946
5003,175.0342
5004,172.2461


,ADX,SMA,RSI,MACD,MACD_Hist,MACD_Signal,open,high,low,close,adjusted close,volume,dividend amount,split coefficient
5000,16.7950,172.4880,50.9353,0.3751,-0.4032,0.7783,172.5300,174.5500,171.9600,172.2300,170.2787,28819653,0.0000,1.0000
5001,16.3877,172.3370,54.0916,0.4254,-0.2823,0.7077,172.5400,173.4700,172.0800,173.0300,171.0696,22211345,0.0000,1.0000
5002,17.2894,172.4020,60.9626,0.6170,-0.0725,0.6896,173.4400,175.3700,173.0500,175.0000,173.0173,23016177,0.0000,1.0000
5003,18.2500,172.3360,57.7916,0.7083,0.0150,0.6933,174.3500,175.6100,173.9300,174.3500,172.3746,20134092,0.0000,1.0000
5004,18.5320,172.2680,57.6890,0.7702,0.0615,0.7087,174.5500,175.0600,173.4100,174.3300,172.3549,21262614,0.0000,1.0000


,Target
5100,191.6060
5101,190.0316
5102,190.1312
5103,188.1781
5104,188.0784


,ADX,SMA,RSI,MACD,MACD_Hist,MACD_Signal,open,high,low,close,adjusted close,volume,dividend amount,split coefficient
5100,34.0115,187.5700,63.2496,3.7918,-0.1978,3.9896,187.6000,188.7500,186.8700,187.9000,187.2414,22514075,0.0000,1.0000
5101,33.1928,187.6760,61.2097,3.5729,-0.3334,3.9063,187.7200,188.0000,186.7800,187.5000,186.8428,18690547,0.0000,1.0000
5102,31.7358,187.5450,57.9396,3.3104,-0.4767,3.7871,187.2200,188.2300,186.1400,186.8700,186.2150,27482793,0.0000,1.0000
5103,31.7906,187.8700,68.0764,3.3358,-0.3611,3.6968,187.9912,190.2600,187.7500,190.2400,189.5732,23442510,0.0000,1.0000
5104,33.3636,188.4220,71.6573,3.4445,-0.2018,3.6464,191.6350,193.4200,191.3500,191.8300,191.1576,26266174,0.0000,1.0000
